<a href="https://colab.research.google.com/github/Sajidcodecrack/Bulk-Detection/blob/main/Bulk_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 33.1/112.6 GB disk)


In [ ]:
!unzip  /content/yolov11.zip -d /content/

Archive:  /content/yolov11.zip
  inflating: /content/README.dataset.txt  
  inflating: /content/README.roboflow.txt  
  inflating: /content/data.yaml      
   creating: /content/test/
   creating: /content/test/images/
 extracting: /content/test/images/20241223_125207_jpg.rf.272cb52f2b125c747d0bed12999c648f.jpg  
 extracting: /content/test/images/20241223_125207_jpg.rf.5534bea825a716bc500a8ecc31bba192.jpg  
 extracting: /content/test/images/20241223_125207_jpg.rf.7744d5115d1bfd7d88bcf8665a3d5fd4.jpg  
 extracting: /content/test/images/20241223_125302_jpg.rf.6ed37a59760cc5d851f8e53e60029aa3.jpg  
 extracting: /content/test/images/20241227_113711_jpg.rf.dc2cc38a3a0f0774f481e81f2746a9ae.jpg  
 extracting: /content/test/images/DSC_1172_jpg.rf.26b89cb106b755f3760b656fc4b1d85d.jpg  
 extracting: /content/test/images/IMG_20250109_131434_jpg.rf.3200754ae84db15ea981cc46333f2793.jpg  
 extracting: /content/test/images/IMG_20250109_131434_jpg.rf.5522c57f8d1d95cecb2b15273707f91b.jpg  
 extracting:

In [ ]:
from collections import Counter
import yaml
import glob
import os

# Load the dataset configuration
with open("/content/data.yaml", "r") as file:
    data = yaml.safe_load(file)

# Get test images and labels directory
test_labels_dir = data["test"].replace("images", "labels")  # Replace 'images' with 'labels'
label_files = glob.glob(os.path.join(test_labels_dir, "*.txt"))

# Initialize class counter
class_counts = Counter()

# Count instances per class
for label_file in label_files:
    with open(label_file, "r") as f:
        for line in f:
            class_id = int(line.split()[0])  # First value is the class index
            class_counts[class_id] += 1

# Print per-class counts
print("Test dataset class distribution:")
for class_id, count in class_counts.items():
    class_name = data["names"][class_id]  # Get class name from YAML file
    print(f"Class {class_id} ({class_name}): {count} instances")


Test dataset class distribution:
Class 0 (Bulk_waste): 61 instances
Class 1 (No_Bulk_waste): 51 instances


In [ ]:
from collections import Counter
import yaml
import glob
import os

# Load the dataset configuration
with open("/content/data.yaml", "r") as file:
    data = yaml.safe_load(file)

# Function to count class instances in a dataset
def count_instances(label_path):
    label_files = glob.glob(os.path.join(label_path, "*.txt"))
    class_counts = Counter()

    for label_file in label_files:
        with open(label_file, "r") as f:
            for line in f:
                class_id = int(line.split()[0])  # First value is the class index
                class_counts[class_id] += 1

    return class_counts

# Get paths for train, valid, and test labels
datasets = {
    "Train": data["train"].replace("images", "labels"),
    "Valid": data["val"].replace("images", "labels"),
    "Test": data["test"].replace("images", "labels")
}

# Count instances per dataset
for dataset_name, label_path in datasets.items():
    class_counts = count_instances(label_path)

    print(f"\n{dataset_name} dataset class distribution:")
    for class_id, count in class_counts.items():
        class_name = data["names"][class_id]  # Get class name from YAML file
        print(f"Class {class_id} ({class_name}): {count} instances")




Train dataset class distribution:
Class 1 (No_Bulk_waste): 972 instances
Class 0 (Bulk_waste): 252 instances

Valid dataset class distribution:
Class 1 (No_Bulk_waste): 54 instances

Test dataset class distribution:
Class 0 (Bulk_waste): 61 instances
Class 1 (No_Bulk_waste): 51 instances


In [ ]:
# Train YOLO11n on COCO8 for 10 epochs
!yolo train model=yolo11n.pt data=/content/data.yaml epochs=100 imgsz=640

Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/content/data.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True,

In [ ]:
!yolo val model=/content/runs/detect/train2/weights/best.pt data=/content/data.yaml split=test



Ultralytics 8.3.75 🚀 Python-3.11.11 torch-2.5.1+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11n summary (fused): 238 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs
val: Scanning /content/test/labels... 112 images, 0 backgrounds, 0 corrupt: 100% 112/112 [00:00<00:00, 2348.21it/s]
val: New cache created: /content/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% 7/7 [00:02<00:00,  2.99it/s]
                   all        112        112      0.997          1      0.995      0.994
            Bulk_waste         61         61          1          1      0.995      0.995
         No_Bulk_waste         51         51      0.995          1      0.995      0.993
Speed: 3.3ms preprocess, 5.6ms inference, 0.0ms loss, 2.8ms postprocess per image
Results saved to runs/detect/val2
💡 Learn more at https://docs.ultralytics.com/modes/val
